<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/logo-bdc.png" align="right" width="64"/>

# <span style="color:#336699">BIG Love Data Day - Sensoriamento Remoto aplicado ao estudo de ecossistemas de água doce</span>
<hr style="border:2px solid #0077b9;">

<div style="text-align: left;">
    <a href="https://nbviewer.jupyter.org/github/brazil-data-cube/code-gallery/blob/master/jupyter/Python/stac/stac-introduction.ipynb"><img src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg" align="center"/></a>
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Resumo.</b> Este Jupyter Notebook é parte do material do Love Data Day organizado pela <em>Base de Informações Georeferenciadas</em> (BIG) do <em>Instituto Nacional de Pesquisas Espaciais</em> (INPE). Nele é feita uma demonstração de modelagem de ecossistemas de água doce, utilizando o serviço SpatioTemporal Asset Catalog (STAC) do INPE.
</div>

<div style="text-align: center;font-size: 90%;">
    Rogerio Flores Júnior<sup><a href="https://orcid.org/0000-0001-6181-2158"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Gilberto R. Queiroz<sup><a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Divisão de Observação da Terra e Geoinformática, Instituto Nacional de Pesquisas Espaciais (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:brazildatacube@inpe.br">brazildatacube@inpe.br</a>
    <br/><br/>
    Data do Minicurso: 03 de junho de 2025
</div>

<br/>




# Contexto
---

O uso de dados de Sensoriamento Remoto para prever parâmetros de qualidade da água (QA) que são opticamente ativos (ou seja, que interagem com a luz) tem sido aplicado a águas oceânicas e costeiras há cerca de 50 anos. Graças à nova geração de sensores com resolução espectral, radiométrica e espacial adequadas (como Landsat, Sentinel-2, etc.), nos últimos 15 anos a comunidade científica começou a aplicar o sensoriamento remoto (SR) em estudos de águas interiores.

O sensoriamento remoto permite prever alguns parâmetros de qualidade da água: sedimentos em suspensão, clorofila-a, ficocianina, matéria orgânica dissolvida, carbono, profundidade do disco de Secchi, turbidez... É uma fonte importante de dados que pode auxiliar biólogos, limnólogos e toda a comunidade de ciências aquáticas na compreensão dos padrões da água.

Neste workshop, vamos aprender como usar dados de Sensoriamento Remoto aplicados às ciências aquáticas. Utilizaremos dados in situ disponíveis no conjunto de dados GLORIA (Lehmann et al. 2023) para gerar um modelo de aprendizado de máquina por arvores de decisão para a determinação da concentração de sólidos totais em Suspensão (TSS).

Assim, com um algoritmo calibrado, aplicaremos o modelo desenvolvido aos dados de Reflectância de Superfície do Sentinel-2/MSI, corrigidos atmosfericamente usando o ACOLITE. Por via de compraração, faremos aplicação em duas data, Antes e após as enchentes no estado do Rio Grande do Sul.

O fluxo de processamento está dividido em três tópicos:

1) Instalação dos pacotes, download dos dados, simulação das bandas e remoção de outliers (etapa de pré-processamento);

2) Desenvolvimento do modelo (treinamento, validação);

3) Aplicação do modelo: aplicação dos algoritmos aos dados de satélite usando o `STAC`

### O que esperamos obter como resultado?

1) Um algoritmo de sólidos totais em Suspensão utilizando um modelo de aprendizado de máquina;

2) Introdução a correção atmosférica de imagens para ambientes aquáticos com o ACOLITE (Vanhellemont and Ruddick, 2018) (https://www.sciencedirect.com/science/article/pii/S0034425718303481);

3) Download e aplicação da correção atmosférica ACOLITE em dados Sentinel-2 L1C obtidos através do STAC do BDC;

4) Comparação dos resultados entre as duas datas de análise.

# Área de Estudo
---
A Bacia Hidrográfica do Lago Guaíba, localizada na Região Hidrográfica da Bacia do Guaíba, possui área de 2.919 km² e população estimada de 1.344.982 habitantes (2020), sendo 1.324.782 habitantes em áreas urbanas e 20.199 habitantes em áreas rurais.

![Figure 01](https://sema.rs.gov.br/upload/recortes/202104/29082641_74342_GDO.png)

# Instalações e configurações
---

Pacotes necessários:

`rasterio`
`pandas`
`geopandas`
`tqdm`
`sklearn`
`scipy`
`pystac`

Para este minicurso vamos utilizar o ambiente python `Geospatial` do BDC que conta estas bibliotecas por padrão.

mas 

Caso deseje efetuar a instalação usando --PyPI-- com `pip`, use os seguintes comandos:

In [ ]:
#!pip install rasterio pandas geopandas tqdm pystac-client sklearn scipy

Caso não tenha o pacote de simulação de bandas, descomentar e executar a célula abaixo:

In [ ]:
#!pip install git+https://github.com/LabISA-INPE/rotina-simulacaobandas-python.git

In [ ]:
# Importando os módulos necessários
import os
# Criando os diretórios
os.makedirs("Data", exist_ok=True)
os.makedirs("Outputs", exist_ok=True)
os.makedirs("Scripts", exist_ok=True)


# Dados
---


## GLORIA Dataset

O conjunto de dados GLORIA é uma compilação de dados de reflectância de sensoriamento remoto (Rrs) e qualidade da água para corpos d’água em escala global, com dados dedicados a ecossistemas de água doce. É gratuito, acessível a todos e cobre grande parte do planeta, com mais de 7.000 amostras (Figura 01).

Vale lembrar que a Reflectância de Sensoriamento Remoto (Rrs) é a razão entre a radiância emergente da água e a irradiância descendente, compensada pela radiância do céu e corrigida pelos efeitos de brilho especular (glint) (Equação 01).

Para mais informações, consulte a publicação [(Lehmann et al. 2023)](https://www.nature.com/articles/s41597-023-01973-y) e o dataset disponível no [PANGAEA](http://https://doi.pangaea.de/10.1594/PANGAEA.948492) e [Nature Earth and Environmment blog post](https://communities.springernature.com/posts/gloria-challenges-in-developing-a-globally-representative-hyperspectral-in-situ-dataset-for-the-remote-sensing-of-water-resources)



![Figure 01](https://earthenvironmentcommunity.nature.com/cdn-cgi/image/metadata=copyright,fit=scale-down,format=auto,sharpen=1,quality=95/https://images.zapnito.com/uploads/hiCMOprnTtSCTJNv78gu_locations.jpg)


In [ ]:
import os
import requests
import zipfile
from pathlib import Path
import pandas as pd

###### Baixar os dados GLORIA ##########
URL = 'https://download.pangaea.de/dataset/948492/files/GLORIA-2022.zip'

# Espera de 300s
TIMEOUT = 300

# Criando o diretório para os dados
if not os.path.exists('Data/GLORIA_2022/'):
    
    # Download
    print("Baixando o GLORIA...")
    response = requests.get(URL, timeout=TIMEOUT)
    response.raise_for_status()  # Raises an HTTPError for bad responses
    
    with open('Data/GLORIA.zip', 'wb') as f:
        f.write(response.content)
    
    # Extraindo os dados
    print("Extraindo o zip file...")
    with zipfile.ZipFile('Data/GLORIA.zip', 'r') as zip_ref:
        zip_ref.extractall('Data')
    
    print("Download e extração completas!")
else:
    print("O diretório de dados do GLORIA já existe.")

In [ ]:
#[CODE - PLOT Gloria]

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Alternative version with even better styling
def plot_gloria_concentrations(rrs_data, 
                                       gloria_ids=['GID_7403', 'GID_1805', 'GID_2468'],
                                       titles=['Alta Clorofila', 'Alto Sedimento', 'Alto aCDOM'],
                                       colors=['#228B22', '#8B4513', '#B8860B'],  # Forest Green, Saddle Brown, Dark Goldenrod
                                       ylims=[(0, 0.06), (0, 0.06), (0, 0.0005)],
                                       wavelength_range=(400, 900),
                                       figsize=(16, 5.5),
                                       title_fontsize=15):
    """
    Enhanced version with better colors and styling.
    """
    
    # Set style
    plt.style.use('default')
    
    # Create wavelength range and column names
    wavelengths = list(range(wavelength_range[0], wavelength_range[1] + 1))
    rrs_cols = [f"Rrs_{wl}" for wl in wavelengths]
    
    # Create 1x3 horizontal subplot grid with more space
    fig, axes = plt.subplots(1, 3, figsize=figsize)
    
    # Plot each condition
    for i, (gid, title, color, ylim) in enumerate(zip(gloria_ids, titles, colors, ylims)):
        # Filter data for current GLORIA_ID
        data = rrs_data[rrs_data['GLORIA_ID'] == gid][rrs_cols]
        
        if not data.empty:
            # Plot all spectra for this condition
            axes[i].plot(wavelengths, data.T, color=color, linewidth=2, alpha=0.9)
            axes[i].set_ylim(ylim)
            axes[i].set_xlim(wavelength_range)
            axes[i].set_title(title, fontsize=title_fontsize, pad=20, fontweight='bold')
            
            # Enhanced grid
            axes[i].grid(True, alpha=0.4, linestyle='-', linewidth=0.5, color='gray')
            
            # Clean styling
            axes[i].spines['top'].set_visible(False)
            axes[i].spines['right'].set_visible(False)
            axes[i].spines['left'].set_linewidth(1.2)
            axes[i].spines['bottom'].set_linewidth(1.2)
            axes[i].spines['left'].set_color('#333333')
            axes[i].spines['bottom'].set_color('#333333')
            
            # Tick styling
            axes[i].tick_params(axis='both', which='major', labelsize=10, 
                              colors='#333333', width=1.2)
            
            # Labels
            axes[i].set_xlabel('Wavelength (nm)', fontsize=12, fontweight='bold', 
                             color='#333333', labelpad=10)
            
            # Only set y-axis label for the leftmost plot
            if i == 0:
                axes[i].set_ylabel('Remote Sensing Reflectance', fontsize=12, 
                                 fontweight='bold', color='#333333', labelpad=10)
        else:
            axes[i].text(0.5, 0.5, f'No data for {gid}', 
                       transform=axes[i].transAxes, ha='center', va='center',
                       fontsize=12, bbox=dict(boxstyle="round,pad=0.5", 
                                            facecolor="lightcoral", alpha=0.7))
            axes[i].set_title(title, fontsize=title_fontsize, pad=20, fontweight='bold')
    
    # Adjust layout with more padding
    plt.tight_layout(pad=3.0)
    
    return fig, axes

In [ ]:
##### Avaliando os dados GLORIA  #######
meta_and_lab = pd.read_csv("Data/GLORIA_2022/GLORIA_meta_and_lab.csv")
rrs = pd.read_csv("Data/GLORIA_2022/GLORIA_Rrs.csv")

##### Plotando os dados GLORIA #######
fig, axes = plot_gloria_concentrations(rrs)
plt.show()


In [ ]:
rrs

# Simulação de Bandas
---

Quando simulamos uma banda de satélite, estamos compensando as diferenças na sensibilidade dos detectores a cada comprimento de onda. A figura abaixo mostra as diferenças na função de resposta espectral para os sensores Sentinel-2A/MSI, Landsat-8/OLI e Landsat-7/ETM+. É possível notar que valores de resposta espectral relativa próximos de "1" indicam que o detector consegue medir (ou detectar) toda a radiância naquele comprimento de onda.

Uma banda de um sensor é composta por um intervalo desses comprimentos de onda e, portanto, a banda simulada é a integração da R[rs] considerando a curva de Resposta Espectral Relativa.


![Figure 02](https://upload.wikimedia.org/wikipedia/commons/7/7d/Spectral_responses_of_Landsat_7_ETM%2B%2C_Landsat_8_OLI_and_Sentinel_2_MSI_in_the_visible_and_near_infrared.png)


In [ ]:
#https://github.com/LabISA-INPE/rotina-simulacaobandas-python

# from Scripts.simulacaoBandas import SateliteBandSimulator

# SBS = SateliteBandSimulator()

# # Select bands between 400 and 900 nm and transpose
# band_columns = [f"Rrs_{wl}" for wl in range(400, 901)]
# spectra_formated = rrs[band_columns].T

# # MSI Simulation
# MSI_sim = SBS.msi(spectra=spectra_formated, point_names=rrs['GLORIA_ID'])
#MSI_sim['s2a'].to_csv('Outputs/S2a_gloria_dataset.csv')

In [ ]:
# lendo os dados csv
MSI_load = pd.read_csv('Outputs/S2a_gloria_dataset.csv')

# Selecionando os dados
MSI = MSI_load.iloc[:, 1:].T

# removendo colunas extras
MSI = MSI.drop("Wave", axis=0)

# Nomeando as bandas de acordo com seu comprimento de onda central
MSI.columns = ['443', '492', '560', '665', '704', '740', '783', '833', '865']

# Criando uma coluna para o nome da cada ponto
MSI["GLORIA_ID"]  = MSI.index
MSI

In [ ]:
#[CODE - Merge and Plot]

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

def process_gloria_data(meta_and_lab, msi_data, 
                       chla_max=1000, chla_min=0,
                       ndci_min=0, ndci_max=2):
    ## Merge with water quality, lat long (By GLORIA_ID)
    # Select specific columns from meta_and_lab (equivalent to R's select)
    meta_selected = meta_and_lab[['GLORIA_ID', 'TSS', 'Latitude', 'Longitude']]
    
    # Merge datasets (equivalent to R's merge)
    merged = pd.merge(meta_selected, msi_data, on='GLORIA_ID', how='inner')
    
    merged = merged[merged['TSS'].notna()]
    
    return merged

def plot_correlation_chart(data, columns=['TSS','704'], 
                          figsize=(10, 8), method='pearson'):
    # Select only the specified columns
    corr_data = data[columns]
    
    # Create correlation matrix
    corr_matrix = corr_data.corr(method=method)
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=figsize)
    
    # Upper triangle: Correlation coefficients
    mask_upper = np.triu(np.ones_like(corr_matrix, dtype=bool))
    
    # Lower triangle: Scatter plots
    for i in range(len(columns)):
        for j in range(len(columns)):
            if i == j:
                # Diagonal: Histograms
                axes[i, j].hist(corr_data.iloc[:, i], bins=20, alpha=0.7, color='skyblue')
                axes[i, j].set_title(f'{columns[i]} Distribution')
                axes[i, j].set_ylabel('Frequency')
            elif i > j:
                # Lower triangle: Scatter plots
                axes[i, j].scatter(corr_data.iloc[:, j], corr_data.iloc[:, i], 
                                 alpha=0.6, s=30)
                axes[i, j].set_xlabel(columns[j])
                axes[i, j].set_ylabel(columns[i])
                
                # Add trend line
                z = np.polyfit(corr_data.iloc[:, j], corr_data.iloc[:, i], 1)
                p = np.poly1d(z)
                axes[i, j].plot(corr_data.iloc[:, j], p(corr_data.iloc[:, j]), 
                               "r--", alpha=0.8)
            else:
                # Upper triangle: Correlation coefficients
                corr_val = corr_matrix.iloc[i, j]
                axes[i, j].text(0.5, 0.5, f'r = {corr_val:.3f}', 
                               transform=axes[i, j].transAxes,
                               fontsize=16, ha='center', va='center')
                axes[i, j].set_xlim(0, 1)
                axes[i, j].set_ylim(0, 1)
                axes[i, j].set_xticks([])
                axes[i, j].set_yticks([])
    
    plt.tight_layout()
    return fig

def simple_correlation_plot(data, x_col='TSS', y_col='704', figsize=(8, 6)):   
    fig, ax = plt.subplots(figsize=figsize)
    
    # Create scatter plot
    ax.scatter(data[x_col], data[y_col], alpha=0.6, s=50)
    
    # Add trend line
    z = np.polyfit(data[x_col], data[y_col], 1)
    p = np.poly1d(z)
    ax.plot(data[x_col], p(data[x_col]), "r--", alpha=0.8, linewidth=2)
    
    # Calculate correlation
    corr_coeff, p_value = pearsonr(data[x_col], data[y_col])
    
    # Add correlation info to plot
    ax.text(0.05, 0.95, f'r = {corr_coeff:.3f}\np = {p_value:.3f}', 
            transform=ax.transAxes, fontsize=12,
            verticalalignment='top', 
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax.set_xlabel(x_col, fontsize=12)
    ax.set_ylabel(y_col, fontsize=12)
    ax.set_title(f'{y_col} vs {x_col}', fontsize=14)
    ax.grid(True, alpha=0.3)
    
    return fig, ax


In [ ]:
meta_and_lab = pd.read_csv("Data/GLORIA_2022/GLORIA_meta_and_lab.csv")
meta_and_lab

In [ ]:
# Lendo os dados
meta_and_lab = pd.read_csv("Data/GLORIA_2022/GLORIA_meta_and_lab.csv")

# Pré-processando os dados (reoganização de colunas e concatenando as tabelas)
merged = process_gloria_data(meta_and_lab, MSI)

# Informações após reoganização
print(f"Original data: {len(meta_and_lab)} rows")
print(f"After processing: {len(merged)} rows")

# Graficos de avaliação
fig2 = plot_correlation_chart(merged)
plt.show()


In [ ]:
# Salvando os dados em csv 
merged.to_csv('Outputs/sentinel2_simulated_filtered_TSS.csv', index=False)

# Modelo Random Forest para Sólidos Totais em Suspenssão
<hr style="border:1px solid #0077b9;">

Agora que nós já fizemos as primeiras organizações nos dados, podemos começar a fazer um treinamento e validação de um algoritmo de Random Forest com os dados de campo simulados

Este código irá utilizar os dados organizados anteriormente para gerar um algoritmo de árvores de decisão para estimativa da concentração de Sólidos Totais em Suspenssão .

Este código é apenas um exemplo. Idealmente, deveríamos validar o modelo aplicado na imagem em dados coletados ao mesmo tempo da passagem do satélite.

Como estes dados não estão disponíveis, iremos treinar e validar utilizando apenas os dados de campo do GLORIA.


In [ ]:
# [CODE - Funções e plots]
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import seaborn as sns


# Calculando as métricas de erro
def calculate_metrics(y_true, y_pred, dataset_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n{dataset_name} Metrics:")
    print(f"RMSE: {rmse:.3f}")
    print(f"MAE: {mae:.3f}")
    print(f"R²: {r2:.3f}")
    
    return rmse, mae, r2

# Gráficos do Random Forest
def plot_rf_results(y_train, y_train_pred, y_test, y_test_pred, 
                   train_r2, train_rmse, test_r2, test_rmse, 
                   feature_importance, target_name='TSS'):
    """
    Plot Random Forest model results including training/test performance,
    feature importance, and residuals.
    
    Parameters:
    -----------
    y_train, y_train_pred : array-like
        Training observed and predicted values
    y_test, y_test_pred : array-like  
        Test observed and predicted values
    train_r2, train_rmse : float
        Training set R² and RMSE metrics
    test_r2, test_rmse : float
        Test set R² and RMSE metrics
    feature_importance : DataFrame
        DataFrame with 'Feature' and 'Importance' columns
    target_name : str
        Name of target variable for plot labels
    """
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Training set: Predicted vs Observed
    axes[0, 0].scatter(y_train, y_train_pred, alpha=0.6, s=20)
    axes[0, 0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2)
    axes[0, 0].set_xlabel(f'Observed {target_name}')
    axes[0, 0].set_ylabel(f'Predicted {target_name}')
    axes[0, 0].set_title(f'Training Set\nR² = {train_r2:.3f}, RMSE = {train_rmse:.3f}')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Test set: Predicted vs Observed
    axes[0, 1].scatter(y_test, y_test_pred, alpha=0.6, s=20, color='orange')
    axes[0, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    axes[0, 1].set_xlabel(f'Observed {target_name}')
    axes[0, 1].set_ylabel(f'Predicted {target_name}')
    axes[0, 1].set_title(f'Test Set\nR² = {test_r2:.3f}, RMSE = {test_rmse:.3f}')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Feature importance plot
    axes[1, 0].barh(feature_importance['Feature'], feature_importance['Importance'])
    axes[1, 0].set_xlabel('Feature Importance')
    axes[1, 0].set_title('Random Forest Feature Importance')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Residuals plot
    residuals_test = y_test - y_test_pred
    axes[1, 1].scatter(y_test_pred, residuals_test, alpha=0.6, s=20, color='green')
    axes[1, 1].axhline(y=0, color='r', linestyle='--')
    axes[1, 1].set_xlabel(f'Predicted {target_name}')
    axes[1, 1].set_ylabel('Residuals')
    axes[1, 1].set_title('Residuals Plot (Test Set)')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import seaborn as sns

# Lendo os dados
data = pd.read_csv("Outputs/sentinel2_simulated_filtered_TSS.csv")
print(f"Original data shape: {data.shape}")
print(f"Column names: {list(data.columns)}")


In [ ]:
np.random.seed(13)

# Definindo as bandas a serem utilizadas
wavelength_bands = ['443', '492', '560', '665', '704', '740', '783', '833', '865']
print(f"Usando as bandas: {wavelength_bands}")

# Filtrando os dados
data_clean = data.dropna(subset=wavelength_bands + ['TSS']).copy()
print(f"Limpando NaNs dos dados: {data_clean.shape}")

# Filtrando valores excessivos (Opcional)
tss_lower, tss_upper = 0.1, 300  # Adjust based on your data range
data_filtered = data_clean[(data_clean['TSS'] >= tss_lower) & 
                          (data_clean['TSS'] <= tss_upper)].copy()
print(f"Filtrando valores entre: {tss_lower}-{tss_upper}): {data_filtered.shape}")

# Separando os dados de X e y
X = data_filtered[wavelength_bands].copy()
y = data_filtered['TSS'].copy()

print(f"Formato Variáveis preditivas: {X.shape}")
print(f"Formato Variável alvo: {y.shape}")

# Criando o conjunto de train/test (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=13)

print(f"Treinamento - X: {X_train.shape}, y: {y_train.shape}")
print(f"Teste - X: {X_test.shape}, y: {y_test.shape}")

# Estatísticas
print(f"\nestatisticas para a variável alvo (TSS):")
print(f"Treinamento - Média: {y_train.mean():.3f}, Std: {y_train.std():.3f}")
print(f"Teste - Média: {y_test.mean():.3f}, Std: {y_test.std():.3f}")

# Modelo Random Forest
print("\n## Treinando o Modelo Random Forest")

# Variável com as Configurações do RF
rf_model = RandomForestRegressor(
    n_estimators=300,        # Número de arvores
    max_depth=40,           # Maxima profundidade das arvores
    min_samples_split=20,    # Minimo de amostras em cada divisão
    min_samples_leaf=4,     # Minimo de amostras em cada folha da arvore
    random_state=13,        # Estado Pseudo_randômico
    n_jobs=-1              # Usar todos os processadores
)

# Fit the model
rf_model.fit(X_train, y_train)
print("Modelo Random Forest treinado com sucesso!")

# Predição
y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)

# Calculate and display metrics
train_rmse, train_mae, train_r2 = calculate_metrics(y_train, y_train_pred, "Training")
test_rmse, test_mae, test_r2 = calculate_metrics(y_test, y_test_pred, "Test")


In [ ]:
# Feature importance
feature_importance = pd.DataFrame({
    'Feature': wavelength_bands,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\nFeature Importance:")
print(feature_importance)


In [ ]:
# Plotar resultados
plot_rf_results(y_train, y_train_pred, y_test, y_test_pred,
               train_r2, train_rmse, test_r2, test_rmse, 
               feature_importance, target_name='TSS')

# Resumo da performance do modelo
print(f"\n{'='*50}")
print(f"RESUMO DO MODELO RANDOM FOREST")
print(f"{'='*50}")
print(f"Características usadas: {len(wavelength_bands)} bandas Rrs")
print(f"Amostras de treinamento: {X_train.shape[0]}")
print(f"Amostras de teste: {X_test.shape[0]}")
print(f"")
print(f"Performance no Conjunto de Teste:")
print(f"  Pontuação R²: {test_r2:.3f}")
print(f"  RMSE: {test_rmse:.3f}")
print(f"  MAE: {test_mae:.3f}")
print(f"")
print(f"Características mais importantes:")
for i, row in feature_importance.head(3).iterrows():
    print(f"  {row['Feature']}: {row['Importance']:.3f}")

In [ ]:
# Salvar os resultados para CSV
results_df = pd.DataFrame({
    'Observed_TSS': y_test,
    'Predicted_TSS': y_test_pred,
    'Residuals': y_test - y_test_pred
})
results_df.to_csv('Outputs/rf_predictions.csv', index=False)
print(f"\nPredictions saved to 'rf_predictions.csv'")


In [ ]:
# Salvar o modelo para posterior aplicação em imagens S2
import joblib

# Salvar o modelo
joblib.dump(rf_model, 'Outputs/random_forest_tss_model.pkl')
print("Modelo salvo como 'random_forest_tss_model.pkl'")

# Salvar nomes das características (importante para aplicação no S2)
feature_names = ['443', '492', '560', '665', '704', '740', '783', '833', '865']
joblib.dump(feature_names, 'Outputs/model_features.pkl')
print("Características salvas como 'model_features.pkl'")

# Salvar metadados do modelo para referência
model_metadata = {
    'target_variable': 'TSS',
    'n_features': len(feature_names),
    'feature_names': feature_names,
    'test_r2': test_r2,
    'test_rmse': test_rmse,
    'test_mae': test_mae,
    'model_params': rf_model.get_params()
}
joblib.dump(model_metadata, 'Outputs/model_metadata.pkl')
print("Metadados do modelo salvos como 'model_metadata.pkl'")

print(f"\nPara aplicar este modelo em imagens S2, você precisará:")
print(f"1. Carregar modelo: rf_model = joblib.load('random_forest_tss_model.pkl')")
print(f"2. Carregar características: features = joblib.load('model_features.pkl')")
print(f"3. Garantir que as bandas Rrs do S2 correspondam às características de treinamento")
print(f"4. Aplicar: predictions = rf_model.predict(s2_rrs_data)")

# Download de imagens L1C e Correção atmosférica para ambientes aquáticos
<hr style="border:1px solid #0077b9;">

Para aplicar a correção do ACOLITE precisamos de duas coisas: Primeiro, precisamos baixar os bundles em nível 1 - sem correção atmosférica. Por sorte, o BDC fornece esses dados pra gente - o que facilita imensamente o acesso aos dados. 

Usaremos o pystac-client para fazer as buscas nos dados - "imagem Sentinel-2/MSI" na data anterior ao Evento () e após as enchentes na data de ()

In [ ]:
from pystac_client import Client
from datetime import datetime
import requests
from tqdm import tqdm
import glob
import os

In [ ]:
# Conectando ao catalogo STAC
stac_obj = Client.open("https://data.inpe.br/bdc/stac/v1/")

# Parâmetros de Busca
datas_antes = "2024-04-18/2024-04-20"
datas_depois = "2024-06-01/2024-06-03"
BBOX = [-51.33503, -30.39062, -51.01744, -30.00016]  # xmin, ymin, xmax, ymax


# Buscando os Items
search_obj = stac_obj.search(
    collections=["S2_L1C_BUNDLE-1"],
    bbox=BBOX,
    datetime=datas_antes,
)
items = list(search_obj.items())

# Itens encontrados
print(f"Found {len(items)} items")
for i, item in enumerate(items):
    print(f"Item {i+1}: {item.id}")
    print(f"  Date: {item.datetime}"),
    if 'eo:cloud_cover' in item.properties:
        print(f"  Cloud cover: {item.properties['eo:cloud_cover']}%")


items[0].assets['asset']


In [ ]:
# Baixar os items
download_dir = 'Outputs'
for item in items:
    print(f"Downloading: {item.id}")
    
    # Pegando o link
    download_url = items[0].assets['asset'].href
    filename = f"{item.id}.zip"
    filepath = os.path.join(download_dir, filename)
    
    # verificando se ja baixado
    if os.path.exists(filepath):
        print(f"File already exists: {filename}")
        continue
    
    # Fazendo o download
    response = requests.get(download_url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    with open(filepath, 'wb') as f, tqdm(desc=filename, total=total_size, unit='B', unit_scale=True) as pbar:
        for chunk in response.iter_content(chunk_size=8192):
            size = f.write(chunk)
            pbar.update(size)
    
    print(f"Baixado a imagem: {filename}")

print(f"\nDownload completo! Arquivos salvos em: {filepath}")

In [ ]:
from pathlib import Path
import zipfile
import os
import glob

# Pasta com os ZIPs extraídos que será criada
pasta_destino = Path("Outputs/Extraidos/")
pasta_destino.mkdir(parents=True, exist_ok=True)  # Se não existir, crio a pasta

# Path base das imagens
path_images = 'Outputs'

# Aqui vou fazer a busca recursiva dos arquivos .ZIP .zip
arquivos_zip = glob.glob(os.path.join(path_images, "**", "*.zip"), recursive=True)
print(arquivos_zip)

# Loop para extrair todos os .zip
for zip_path in arquivos_zip:
    nome_arquivo = Path(zip_path).stem  # Nome do zip sem extensão
    print(nome_arquivo)
    
    # Extrair diretamente para a pasta destino (sem criar subpasta)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(pasta_destino)  # Extrai diretamente para pasta_destino
    
    print(f"Extraído: {zip_path} → {pasta_destino}")


In [ ]:
# [CODE - Leitura das bandas da Imagem processada]

import numpy as np
from rasterio.windows import from_bounds, Window
import rasterio
from matplotlib import pyplot as plt
from rasterio.warp import transform_bounds
from rasterio.windows import transform as window_transform
plt.rcParams['figure.figsize'] = (20, 10)

def read_img(uri: str, bbox: tuple = None, bbox_crs: str = "EPSG:4326", masked: bool = True):
    """Read raster window as numpy.ma.masked_array."""
    with rasterio.open(uri) as src:
        if bbox is not None:
            # reprojeta o bbox para o CRS do raster
            projected_bbox = transform_bounds(bbox_crs, src.crs, *bbox)
            window = from_bounds(*projected_bbox, transform=src.transform)
            data = src.read(window=window, masked=masked)
            new_transform = rasterio.windows.transform(window, src.transform)

            profile = src.profile.copy()
            profile.update({
                "height": window.height,
                "width": window.width,
                "transform": new_transform
            })
        else:
            data = src.read(masked=masked)
            profile = src.profile

    return data, profile

def normalize_and_adjust_brightness(array, brightness_factor=1):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    normalized = (array - array_min) / (array_max - array_min)
    brightened = np.clip(normalized * brightness_factor, 0.0, 1.0)
    return brightened



In [ ]:
b2 = 'Outputs/Extraidos/S2A_MSIL1C_20240418T132241_N0510_R038_T22JDM_20240418T164038.SAFE/GRANULE/L1C_T22JDM_A046081_20240418T132235/IMG_DATA/T22JDM_20240418T132241_B02.jp2'
b3 = 'Outputs/Extraidos/S2A_MSIL1C_20240418T132241_N0510_R038_T22JDM_20240418T164038.SAFE/GRANULE/L1C_T22JDM_A046081_20240418T132235/IMG_DATA/T22JDM_20240418T132241_B03.jp2'
b4 = 'Outputs/Extraidos/S2A_MSIL1C_20240418T132241_N0510_R038_T22JDM_20240418T164038.SAFE/GRANULE/L1C_T22JDM_A046081_20240418T132235/IMG_DATA/T22JDM_20240418T132241_B04.jp2'

BBOX = [-51.33503, -30.39062, -51.01744, -30.00016]
b02_image, _ = read_img(b2,BBOX)
b03_image, _ = read_img(b3,BBOX)
b04_image, _ = read_img(b4,BBOX)

rgb_normalized_stack = np.dstack((
    normalize_and_adjust_brightness(b04_image[0], 8.5), 
    normalize_and_adjust_brightness(b03_image[0], 10.5), 
    normalize_and_adjust_brightness(b02_image[0], 8.5)))
plt.imshow(rgb_normalized_stack)

In [ ]:
b2 = 'Outputs/Extraidos/S2B_MSIL1C_20240602T132239_N0510_R038_T22JDM_20240602T134842.SAFE/GRANULE/L1C_T22JDM_A037816_20240602T132233/IMG_DATA/T22JDM_20240602T132239_B02.jp2'
b3 = 'Outputs/Extraidos/S2B_MSIL1C_20240602T132239_N0510_R038_T22JDM_20240602T134842.SAFE/GRANULE/L1C_T22JDM_A037816_20240602T132233/IMG_DATA/T22JDM_20240602T132239_B03.jp2'
b4 = 'Outputs/Extraidos/S2B_MSIL1C_20240602T132239_N0510_R038_T22JDM_20240602T134842.SAFE/GRANULE/L1C_T22JDM_A037816_20240602T132233/IMG_DATA/T22JDM_20240602T132239_B04.jp2'

BBOX = [-51.33503, -30.39062, -51.01744, -30.00016]
b02_image, _ = read_img(b2,BBOX)
b03_image, _ = read_img(b3,BBOX)
b04_image, _ = read_img(b4,BBOX)

rgb_normalized_stack = np.dstack((
    normalize_and_adjust_brightness(b04_image[0], 6.5), 
    normalize_and_adjust_brightness(b03_image[0], 10.5), 
    normalize_and_adjust_brightness(b02_image[0], 10.5)))
plt.imshow(rgb_normalized_stack)

<img src="https://camo.githubusercontent.com/819c228f84e8da719975991335b2cd2fc045dc0b182adee76f7c3277c2f3259e/68747470733a2f2f6173736574732e6769736875622e6f72672f696d616765732f6879706572636f6173745f6c6f676f5f3630302e706e67" align="right" width="200"/>

# Rodar o ACOLITE em Python 
---


Para rodar o ACOLITE iremos utilizar como auxílio o pacote hypercoast, que é um pacote desenvolvido por Bingqing Liu e Qiusheng Wu (disponível em: https://hypercoast.org/#license). Ele é feito inicialmente para trabalhar com dados hiperesepctrais de sensoriamento remoto - mas com diversas aplicações importantes com dados multiespectrais - incluindo maneiras mais simples de instalar o ACOLITE e rodar ele em ambiente Jupyter Notebook - o que não é totalmente trivial. 



In [ ]:
# Se você ainda não rodou este comando, você vai precisar baixar  o ACOLITE e instalar o HYPERCOAST!!!!!! Desmarque o # nessa linha


#!git clone --depth 1 https://github.com/acolite/acolite
#!pip install "hypercoast[extra]"

In [ ]:
import hypercoast

In [ ]:
acolite_dir = hypercoast.download_acolite('Scripts/acolite/')
out_dir = os.path.join('Outputs', "acolite_results")

In [ ]:
input_dir = os.path.join('Outputs/', "Extraidos")
input_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir)]
input_files

In [ ]:
# for i in input_files:

#     output_dir = str.replace(i, 'Extraidos', "ACOLITE")
    
#     hypercoast.run_acolite(
#         acolite_dir=acolite_dir,
#         input_file=i,
#         out_dir=output_dir,
#         l2w_parameters="Rrs_*",
#         rgb_rhot=True,
#         rgb_rhos=True,
#         map_l2w=True,
#         polygon = 'Data/Area_Lago_Guaiba.geojson', #Polígono do lago
#         l2w_export_geotiff = True) # Exporta o L2W em Geotiff

In [ ]:
b3 = 'Outputs/ACOLITE/S2A_MSIL1C_20240418T132241_N0510_R038_T22JDM_20240418T164038.SAFE/S2A_MSI_2024_04_18_13_30_54_T22JDM_L2W_Rrs_443.tif'
b4 = 'Outputs/ACOLITE/S2A_MSIL1C_20240418T132241_N0510_R038_T22JDM_20240418T164038.SAFE/S2A_MSI_2024_04_18_13_30_54_T22JDM_L2W_Rrs_560.tif'
b5 = 'Outputs/ACOLITE/S2A_MSIL1C_20240418T132241_N0510_R038_T22JDM_20240418T164038.SAFE/S2A_MSI_2024_04_18_13_30_54_T22JDM_L2W_Rrs_665.tif'

b03_image, _ = read_img(b3)
b04_image, _ = read_img(b4)
b05_image, _ = read_img(b5)

rgb_normalized_stack = np.dstack((
    normalize_and_adjust_brightness(b05_image[0], 1.5), 
    normalize_and_adjust_brightness(b04_image[0], 1.5), 
    normalize_and_adjust_brightness(b03_image[0], 1.5)))
plt.imshow(rgb_normalized_stack)



# Aplicação do modelo de TSS e comparação entre as datas
---

Agora que conseguimos executar a correção atmosférica com o ACOLITE, podemos aplicar o modelo de TSS nas imagens e avaliar os resultados para as duas datas


In [ ]:
import pandas as pd
import numpy as np
import rasterio
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
import warnings
import joblib
import pickle
from rasterio.plot import show
from rasterio.warp import reproject, Resampling
from rasterio.transform import from_bounds
import matplotlib.patches as patches

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (20, 10)

In [ ]:
caminho_img_anterior = 'Outputs/ACOLITE/S2A_MSIL1C_20240418T132241_N0510_R038_T22JDM_20240418T164038.SAFE'

caminho_img_depois =  'Outputs/ACOLITE/S2B_MSIL1C_20240602T132239_N0510_R038_T22JDM_20240602T134842.SAFE'

In [ ]:
# [CODE - Carregar as imagens]

import os
import glob
import rasterio
import numpy as np
import joblib

def load_s2_for_rf(image_path, model_path='Outputs/random_forest_tss_model.pkl', features_path='Outputs/model_features.pkl',sensor='S2A'):
    """
    Load Sentinel-2 Rrs bands and prepare them for Random Forest prediction.
    
    Parameters:
    -----------
    image_path : str
        Path to folder containing Rrs band files
    model_path : str
        Path to saved Random Forest model (default: 'random_forest_tss_model.pkl')
    features_path : str
        Path to saved model features (default: 'model_features.pkl')
    
    Returns:
    --------
    rrs_reshaped : numpy.ndarray
        Reshaped Rrs data ready for RF prediction (pixels, bands)
    valid_mask : numpy.ndarray
        Boolean mask for valid pixels
    spatial_info : dict
        Dictionary containing profile, transform, crs, height, width for output
    """
    
    # Load the trained model and required features
    rf_model = joblib.load(model_path)
    model_features = joblib.load(features_path)
    if sensor == 'S2B':
     model_features = ['442', '492', '559', '665', '704', '739', '780', '833', '864']
    print(f"Model expects these wavelengths: {model_features}")
    
    # Find all Rrs band files
    rrs_pattern = os.path.join(image_path, "*_Rrs_*.tif")
    rrs_files = glob.glob(rrs_pattern)
    print(f"Found {len(rrs_files)} Rrs bands:")
    for file in rrs_files:
        print(f"  - {os.path.basename(file)}")
    
    # Load bands in the order required by the model
    rrs_bands_ordered = []
    available_wavelengths = []
    
    for required_wavelength in model_features:
        # Look for file with this wavelength
        wavelength_found = False
        
        for rrs_file in rrs_files:
            # Extract wavelength from filename (e.g., "Rrs_665.tif" -> "665") 
            file_wavelength = os.path.basename(rrs_file).split('_Rrs_')[1].split('.tif')[0]
            
            if file_wavelength == required_wavelength:
                print(f"✓ Loading {required_wavelength}nm from {os.path.basename(rrs_file)}")
                
                with rasterio.open(rrs_file) as src:
                    band_data = src.read(1)
                    rrs_bands_ordered.append(band_data)
                    available_wavelengths.append(required_wavelength)
                    
                    # Save spatial info from first band
                    if len(rrs_bands_ordered) == 1:
                        profile = src.profile
                        transform = src.transform
                        crs = src.crs
                        height, width = band_data.shape
                
                wavelength_found = True
                break
        
        if not wavelength_found:
            print(f"✗ Missing required wavelength: {required_wavelength}nm")
    
    # Check if we have all required bands
    if len(rrs_bands_ordered) != len(model_features):
        print(f"Warning: Missing {len(model_features) - len(rrs_bands_ordered)} required bands!")
        print(f"Available: {available_wavelengths}")
        print(f"Required: {model_features}")
        raise ValueError("Not all required wavelength bands are available!")
    
    # Stack bands in correct order for RF model
    rrs_stack = np.stack(rrs_bands_ordered, axis=0)  # Shape: (n_bands, height, width)
    print(f"Rrs stack shape: {rrs_stack.shape}")
    
    # Reshape for model prediction: (pixels, bands)
    n_bands, height, width = rrs_stack.shape
    rrs_reshaped = rrs_stack.reshape(n_bands, -1).T  # Shape: (n_pixels, n_bands)
    print(f"Reshaped for prediction: {rrs_reshaped.shape}")
    
    # Create mask for valid pixels (no NaN or negative values)
    valid_mask = np.all(np.isfinite(rrs_reshaped) & (rrs_reshaped >= 0), axis=1)
    print(f"Valid pixels: {valid_mask.sum()}/{len(valid_mask)} ({100*valid_mask.mean():.1f}%)")
    
    # Package spatial information
    spatial_info = {
        'profile': profile,
        'transform': transform,
        'crs': crs,
        'height': height,
        'width': width
    }
    
    print(f"Data ready for Random Forest prediction!")
    
    return rrs_reshaped, valid_mask, spatial_info



In [ ]:
rrs_data_anterior, valid_mask_anterior, spatial_info_anterior = load_s2_for_rf(caminho_img_anterior,sensor='S2A')

In [ ]:
rrs_data_depois, valid_mask_depois, spatial_info_depois = load_s2_for_rf(caminho_img_depois,sensor='S2B')

In [ ]:
valid_mask_depois.shape

In [ ]:
# [CODE - Aplicar o RF e plotar]

import matplotlib.pyplot as plt
import rasterio
import numpy as np
import joblib

def apply_rf_model(rrs_data, valid_mask, spatial_info):
    """
    Apply Random Forest model to predict TSS.
    
    Parameters:
    -----------
    rrs_data : numpy.ndarray
        Reshaped Rrs data from load_s2_for_rf
    valid_mask : numpy.ndarray
        Valid pixel mask from load_s2_for_rf
    spatial_info : dict
        Spatial information from load_s2_for_rf
    
    Returns:
    --------
    tss_map : numpy.ndarray
        2D array with TSS predictions
    """
    
    # Load model and apply
    rf_model = joblib.load('Outputs/random_forest_tss_model.pkl')
    
    # Apply Random Forest model to predict TSS
    tss_predictions = np.full(len(valid_mask), np.nan)
    valid_predictions = rf_model.predict(rrs_data[valid_mask])
    tss_predictions[valid_mask] = valid_predictions
    
    # Reshape to image
    tss_map = tss_predictions.reshape(spatial_info['height'], spatial_info['width'])
    
    print(f"TSS prediction complete!")
    valid_tss = tss_map[np.isfinite(tss_map)]
    print(f"TSS range: {valid_tss.min():.1f} - {valid_tss.max():.1f} mg/L")
    print(f"Mean TSS: {valid_tss.mean():.1f} mg/L")
    
    return tss_map

def plot_tss_map(tss_map, title="TSS Map"):
    """
    Plot TSS map and histogram.
    
    Parameters:
    -----------
    tss_map : numpy.ndarray
        2D TSS array from apply_rf_model
    title : str
        Title for the plot
    """
    
    valid_tss = tss_map[np.isfinite(tss_map)]
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # TSS Map
    vmin, vmax = 0, 125  # Fixed range 0-125 mg/L
    im = axes[0].imshow(tss_map, cmap='RdYlBu_r', vmin=vmin, vmax=vmax)
    axes[0].set_title(f'{title} (mg/L)')
    axes[0].axis('off')
    plt.colorbar(im, ax=axes[0], shrink=0.8)
    
    # Histogram
    axes[1].hist(valid_tss, bins=50, alpha=0.7, edgecolor='black')
    axes[1].set_xlabel('TSS (mg/L)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('TSS Distribution')
    axes[1].set_xlim(vmin, vmax)  # Limit histogram x-axis to 0-125
    
    # Add mean and median lines
    mean_tss = valid_tss.mean()
    median_tss = np.median(valid_tss)
    
    axes[1].axvline(mean_tss, color='red', linestyle='--', linewidth=2, 
                   label=f'Média: {mean_tss:.1f}')
    axes[1].axvline(median_tss, color='blue', linestyle='--', linewidth=2, 
                   label=f'Mediana: {median_tss:.1f}')
    
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def save_tss_map(tss_map, spatial_info, output_path):
    """
    Save TSS map as GeoTIFF.
    
    Parameters:
    -----------
    tss_map : numpy.ndarray
        2D TSS array from apply_rf_model
    spatial_info : dict
        Spatial information from load_s2_for_rf
    output_path : str
        Path for output file
    """
    
    tss_profile = spatial_info['profile'].copy()
    tss_profile.update({'count': 1, 'dtype': 'float32', 'nodata': np.nan})
    
    with rasterio.open(output_path, 'w', **tss_profile) as dst:
        dst.write(tss_map.astype(np.float32), 1)
    
    print(f"TSS map saved to: {output_path}")


In [ ]:
# Apply RF model
tss_map_anterior = apply_rf_model(rrs_data_anterior, valid_mask_anterior, spatial_info_anterior)

# Save the image
save_tss_map(tss_map_anterior, spatial_info_anterior, "Outputs/tss_anterior.tif")

# Plot results
plot_tss_map(tss_map_anterior, "TSS Anterior")

In [ ]:
# Apply RF model
tss_map_depois = apply_rf_model(rrs_data_depois, valid_mask_depois, spatial_info_depois)

# Save the image
save_tss_map(tss_map_depois, spatial_info_depois, "Outputs/tss_depois.tif")

# Plot results
plot_tss_map(tss_map_depois, "TSS Depois")